<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">

<h3 align="left"><font color='#DEB887'>💡 Notes:</font></h3>


**userid**
* A unique number that identifies each player.
 
**version**
* Whether the player was put in the control group (gate_30 - a gate at level 30) or the group with the moved gate (gate_40 - a gate at level 40).

**sum_gamerounds**
* The number of game rounds played by the player during the first 14 days after install.
 
**retention_1**
* Did the player come back and play 1 day after installing?

**retention_7**
* Did the player come back and play 7 days after installing?



 ### Libraries imported

In [ ]:

import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [ ]:
data=pd.read_csv("/kaggle/input/mobile-games-ab-testing/cookie_cats.csv")
data.head()


In [ ]:
df=data.copy()


 ## eda

In [ ]:
def check_data(data,head=5):
    print(data.describe().T)
    print("*******************************************")
    #shape
    print(data.shape)
    print("**********************************")
    #data info
    print(data.info())
    print("***************valuecounts************")
    #value counts
    print(data.value_counts().head())
    print("*********na***********")
    print(data.isnull().sum())
    print("********data types******")
    print(data.dtypes)




check_data(df)


Our control group is gate 30 and test one is gate40,variables are retentions which are retention_1 and retention_7,sumrounds

In [ ]:
def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)

    if plot:
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show(block=True)
num_summary(df,"sum_gamerounds",)

Outliers are here in sum_gamerounds

In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit
low,up=outlier_thresholds(df,"sum_gamerounds")

low,up

In [ ]:
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [ ]:
check_outlier(df,"sum_gamerounds")



In [ ]:
def replace_with_thresholds(dataframe, variable, q1=0.05, q3=0.95):
    low_limit, up_limit = outlier_thresholds(dataframe, variable, q1=0.05, q3=0.95)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
#replace


replace_with_thresholds(df,"sum_gamerounds")

In [ ]:
df.sum_gamerounds

In [ ]:
sns.boxplot(df['sum_gamerounds'])


In [ ]:
df.groupby("version").agg({"retention_1":["mean","sum"]})



In [ ]:
df.groupby("version").agg({"retention_7":["mean","sum"]})

In [ ]:
# shapiro
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_30", "retention_1"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# shapiro
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_40", "retention_1"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))



In [ ]:
test_stat, pvalue = levene(df.loc[df["version"] == "gate_30", "retention_1"],
                           df.loc[df["version"]=="gate_40","retention_1"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


### Version change isnt worked,retention are decreasing,Our actions must be taken action to gain users

## Let's see sum_gamerounds

In [ ]:
sms.DescrStatsW(df["sum_gamerounds"]).tconfint_mean()


In [ ]:
test_stat, pvalue = mannwhitneyu(df.loc[df["version"] == "gate_30", "retention_1"],
                                 df.loc[df["version"] == "gate_40", "retention_1"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


In [ ]:
df.groupby("version")["sum_gamerounds"].mean()

In [ ]:
test_stat, pvalue = shapiro(df.loc[df["version"] == "gate_30", "sum_gamerounds"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

test_stat, pvalue = shapiro(df.loc[df["version"]=="gate_40","sum_gamerounds"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


In [ ]:
test_stat, pvalue = levene(df.loc[df["version"] == "gate_30", "sum_gamerounds"],
                           df.loc[df["version"]=="gate_40","sum_gamerounds"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

H0 is unrejected reason why HO>0005

In [ ]:
test_stat, pvalue = mannwhitneyu(df.loc[df["version"] == "gate_30", "sum_gamerounds"],
                              df.loc[df["version"] == "gate_40", "sum_gamerounds"])  

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

<div class="alert alert-block alert-info"> 📌 HO cant be rejected,it means that secomd version doesnt change  affect number of the round.</div>


